In [1]:
import psycopg2 as pg
import pandas as pd
from multiprocessing import Pool

from equities_data import create_feature_df
from config import DATABASE_URI

In [2]:
# Select all tickers that are in US dollars, are common stock, and have at least five years
# of price and fundamental data
sql = """
SELECT
    ticker
FROM
    tickers
WHERE
    (
        EXTRACT(
            YEAR
            FROM
                lastpricedate
        ) - EXTRACT(
            YEAR
            FROM
                firstpricedate
        ) >= 5
        AND EXTRACT(
            YEAR
            FROM
                lastquarter
        ) - EXTRACT(
            YEAR
            FROM
                firstquarter
        ) >= 5
        AND category = 'Domestic Common Stock'
        AND currency = 'USD'
        AND EXCHANGE != 'OTC'
    );
"""

with pg.connect(DATABASE_URI) as conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        results = cur.fetchall()

tickers = [n[0] for n in results]

In [3]:
with Pool(processes=4) as pool:
    pool.map(create_feature_df, tickers)

/home/eadains/EquitiesTrading/equities_data.py:188: RuntimeWarning: divide by zero encountered in log
  result["assets"] = np.log(df.iloc[-1]["assets"]) - np.log(df.iloc[0]["assets"])
/home/eadains/EquitiesTrading/equities_data.py:182: RuntimeWarning: invalid value encountered in double_scalars
  (df.iloc[-1]["assetsc"] - df.iloc[-1]["cash"] - df.iloc[-1]["liabilitiesc"])
/home/eadains/EquitiesTrading/equities_data.py:182: RuntimeWarning: divide by zero encountered in double_scalars
  (df.iloc[-1]["assetsc"] - df.iloc[-1]["cash"] - df.iloc[-1]["liabilitiesc"])
/home/eadains/EquitiesTrading/equities_data.py:182: RuntimeWarning: divide by zero encountered in double_scalars
  (df.iloc[-1]["assetsc"] - df.iloc[-1]["cash"] - df.iloc[-1]["liabilitiesc"])
/home/eadains/EquitiesTrading/equities_data.py:182: RuntimeWarning: divide by zero encountered in double_scalars
  (df.iloc[-1]["assetsc"] - df.iloc[-1]["cash"] - df.iloc[-1]["liabilitiesc"])
/home/eadains/EquitiesTrading/equities_data.py:18

In [9]:
import glob

In [12]:
files = glob.glob("./csv/*.csv")

In [19]:
test = pd.concat((pd.read_csv(f) for f in files))

In [20]:
test

,date,logsize,pb,momentum,issuance,accruals,roa,assets,divyield,beta,stddev,turnover,debt_price,sales_price,monthly_ret,forward_ret,ticker
0,1999-03-31,16.874132,0.410,0.038653,-0.031649,0.163628,-0.037,0.121861,0.000,0.553149,0.088627,0.004154,1.006424,4.797219,-0.070313,0.025210,DECK
1,1999-04-30,16.899029,0.410,-0.046875,-0.031649,0.163628,-0.037,0.121861,0.000,0.655766,0.091348,0.004214,0.981676,4.679254,0.025210,0.249727,DECK
2,1999-05-28,17.199236,0.444,0.375750,-0.023905,-0.215031,-0.029,0.141731,0.000,0.607071,0.092053,0.004309,1.217792,3.664638,0.249727,0.149712,DECK
3,1999-06-30,17.243612,0.444,0.402810,-0.023905,-0.215031,-0.029,0.141731,0.000,0.720143,0.096281,0.004560,1.164933,3.505572,0.149712,-0.078464,DECK
4,1999-07-30,17.161899,0.444,0.059501,-0.023905,-0.215031,-0.029,0.141731,0.000,0.626191,0.095559,0.004522,1.264121,3.804054,-0.078464,-0.148444,DECK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,2021-03-31,26.525491,20.134,0.027927,0.012987,10.171287,0.166,0.508490,0.001,1.418863,0.029202,0.017440,0.022949,0.050372,-0.004009,0.124457,NVDA
235,2021-04-30,26.642793,20.134,0.119949,0.012987,10.171287,0.166,0.508490,0.001,1.452038,0.027429,0.016609,0.020409,0.044797,0.124457,0.060019,NVDA
236,2021-05-28,26.726691,20.840,0.262029,0.012924,5.010517,0.191,0.280908,0.001,1.553452,0.027289,0.015705,0.018782,0.047568,0.060019,0.230111,NVDA
237,2021-06-30,26.934794,20.840,0.332958,0.012924,5.010517,0.191,0.280908,0.001,1.743343,0.026953,0.015769,0.015253,0.038631,0.230111,-0.025171,NVDA
